## Tech Adoption Scenario

Background: A technology company is implementing an AI-powered system to help product managers and marketing teams identify suitable users for targeted product recommendations. The company wants to move beyond basic demographic targeting to a more nuanced approach that can understand natural language queries and user characteristics holistically.

Use Case: Product managers and marketers can use natural language to find users matching specific profiles or needs. For example, they might search for "early adopters with high technical proficiency interested in smart home technology" or "budget-conscious users who prefer Android devices".

## Implementation Overview

This notebook demonstrates how to build and deploy a machine learning model that uses natural language processing to match tech user profiles with product managers' queries. The system leverages:

1. **Sentence Transformers**: To convert both user profiles and natural language queries into semantic vector embeddings
2. **MLflow**: For model packaging, versioning, and deployment
3. **Semantic Search**: Using cosine similarity to find the most relevant user matches

When deployed, product managers can simply type queries like "Find users who are early adopters of smart home technology" and get a list of the most relevant users without needing to construct complex database queries.

## Process Description

This Tech User Similarity Model provides a semantic search capability over user profiles. It works by:

Data Preparation: User data (demographics, tech preferences, adoption patterns) is processed and organized.

Embedding Generation: The SentenceTransformer model converts user profiles into numerical embeddings (high-dimensional vectors) that capture semantic meaning.

MLflow Deployment: The model, embeddings, and dataset are packaged and deployed using MLflow for reproducible inference.

Query Processing: When a product manager enters a natural language query, it's converted to an embedding using the same model.

Similarity Matching: The system calculates cosine similarity between the query embedding and all user embeddings to find the most similar users.

Result Presentation: Top matching users are presented with their relevant information and similarity scores.

## Benefits

Targeted Marketing: Identify users most likely to adopt new technologies based on their profile similarity to successful past campaigns.

Product Development: Understand user segments and their technology preferences to guide product development.

Cross-Selling Opportunities: Find users similar to those who have already adopted specific products.

Natural Language Interface: Allows product managers to search users without needing complex SQL queries or predefined segments.

Scalability: The system can handle millions of user profiles efficiently due to the vector-based search approach.

This system bridges the gap between rich user data and actionable insights by providing an intuitive way to explore user segments and identify targeted opportunities for engagement.

## Technical Implementation

The following code implements the complete Tech User Similarity system. We'll start by importing necessary libraries and setting up our model environment.

In [1]:
# Imports
import os
import json
import torch
import numpy as np
import pandas as pd
from tabulate import tabulate
import mlflow
import mlflow.pyfunc

from mlflow import MlflowClient
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec, TensorSpec, ParamSchema, ParamSpec
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Model settings - using sentence-transformers
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

2025-04-04 20:58:35.067628: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-04 20:58:35.352762: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743800315.457535    1331 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743800315.489504    1331 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743800315.752758    1331 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

## Model Loading

First, we define a function to load our sentence transformer model, which will handle the semantic encoding of text. This model converts text into high-dimensional vectors where similar meanings are positioned closer together in the vector space.

In [2]:
def load_model():
    """Load the sentence-transformers model for embedding"""
    print(f"Loading SentenceTransformer model: {MODEL_NAME}")
    
    # Set device to GPU if available, otherwise use CPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Load the model
    model = SentenceTransformer(MODEL_NAME)
    model.to(device)
    return model

## Model Architecture

The core of our system is the `TechAdoptionSimilarityModel` class which:
1. Loads pre-computed user embeddings and tech data
2. Handles query encoding and similarity computation
3. Formats results for display
4. Includes MLflow integration for model deployment

This class inherits from `mlflow.pyfunc.PythonModel` to make it deployable through MLflow's model registry.

In [3]:
class TechAdoptionSimilarityModel(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        """Load precomputed embeddings, tech data, and sentence-transformer model."""
        # Load precomputed embeddings
        self.embeddings = np.load(context.artifacts['embeddings_path'])
        
        # Load tech dataset
        self.user_data = pd.read_csv(context.artifacts['tech_dataset_path'])
        
        # Print diagnostics about the loaded data
        print(f"Loaded embeddings shape: {self.embeddings.shape}")
        print(f"Loaded tech data shape: {self.user_data.shape}")
        
        # Create user descriptions for reference
        self.user_descriptions = []
        for _, row in self.user_data.iterrows():
            desc = (
                f"User ID: {row['user_id']}, "
                f"Age: {row['age']}, "
                f"Gender: {row['gender']}, "
                f"Location: {row['location']}, "
                f"Education: {row['education']}, "
                f"Job Sector: {row['job_sector']}, "
                f"Annual Income: ${row['annual_income']}, "
                f"Adopter Category: {row['adopter_category']}, "
                f"Tech Interest: {row['tech_interest']}, "
                f"Technical Proficiency: {row['technical_proficiency']}"
            )
            self.user_descriptions.append(desc)
        
        # Load model
        self.model_name = MODEL_NAME
        self.model = SentenceTransformer(self.model_name)
        
        # Set device for model
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"SentenceTransformer model '{self.model_name}' loaded successfully")

    def generate_query_embedding(self, query):
        """Generate embedding for the input query"""
        return self.model.encode(query, convert_to_numpy=True)

    def predict(self, context, model_input, params=None):
        """Find similar tech users based on semantic similarity to the query text"""
        # Extract the query string from model input
        try:
            if hasattr(model_input, "query"):
                if hasattr(model_input["query"], "iloc") or hasattr(model_input["query"], "loc"):
                    query = model_input["query"].iloc[0]
                    if isinstance(query, list):
                        query = query[0]
                elif isinstance(model_input["query"], list):
                    query = model_input["query"][0]
                    if isinstance(query, list):
                        query = query[0]
                else:
                    query = model_input["query"]
            else:
                query = str(model_input)
            
            if not isinstance(query, str):
                query = str(query)
            
        except Exception as e:
            print(f"Error extracting query: {e}")
            print(f"Model input structure: {type(model_input)}")
            print(f"Model input content: {model_input}")
            query = ""
        
        print(f"Processing query: '{query}'")
        query_lower = query.lower()
        
        # Extract parameters
        top_n = params.get("top_n", 5) if params else 5
        
        # Get initial candidates - get more than needed for boosting/reranking
        query_embedding = self.generate_query_embedding(query)
        similarities = cosine_similarity([query_embedding], self.embeddings)[0]
        candidate_indices = np.argsort(similarities)[::-1][:min(top_n * 3, len(self.user_data))]
        
        # --- SCORE ADJUSTMENT LOGIC ---
        # Create mappings for categorical values
        proficiency_values = {
            "Very Low": 1, "Low": 2, "Moderate": 3, 
            "High": 4, "Very High": 5
        }
        
        adopter_values = {
            "Innovator": 5, "Early Adopter": 4, "Early Majority": 3,
            "Late Majority": 2, "Laggard": 1
        }
        
        # Initialize boost array
        sim_boost = np.zeros(len(candidate_indices))
        
        # 1. Technical proficiency boost (up to 0.3)
        if any(term in query_lower for term in ["high proficiency", "tech savvy", "technical users", 
                                               "advanced users", "experienced", "expert"]):
            print("Applying technical proficiency boost")
            prof_scores = []
            for idx in candidate_indices:
                user = self.user_data.iloc[idx]
                prof = proficiency_values.get(user['technical_proficiency'], 0)
                prof_scores.append(prof)
            
            prof_scores = np.array(prof_scores)
            prof_boost = np.clip((prof_scores - 3) / 5, 0, 0.3)  # Up to 0.3 boost for high proficiency
            sim_boost += prof_boost
        
        elif any(term in query_lower for term in ["beginner", "novice", "new to tech", "low proficiency"]):
            print("Applying beginner-friendly boost")
            prof_scores = []
            for idx in candidate_indices:
                user = self.user_data.iloc[idx]
                prof = proficiency_values.get(user['technical_proficiency'], 0)
                prof_scores.append(prof)
            
            prof_scores = np.array(prof_scores)
            # Boost for lower proficiency (reverse scale)
            prof_boost = np.clip((3 - prof_scores) / 5, 0, 0.3)
            sim_boost += prof_boost
        
        # 2. Adopter category boost (up to 0.3)
        if any(term in query_lower for term in ["early adopter", "innovator", "cutting edge", 
                                               "technology enthusiast", "first to adopt"]):
            print("Applying early adopter boost")
            adopter_scores = []
            for idx in candidate_indices:
                user = self.user_data.iloc[idx]
                score = adopter_values.get(user['adopter_category'], 0)
                adopter_scores.append(score)
            
            adopter_scores = np.array(adopter_scores)
            adopter_boost = np.clip((adopter_scores - 2) / 5, 0, 0.3)  # Up to 0.3 boost for early adopters
            sim_boost += adopter_boost
        
        elif any(term in query_lower for term in ["mainstream user", "typical user", "average consumer", "late majority"]):
            print("Applying mainstream user boost")
            # Different mapping for mainstream users (Early/Late Majority get highest scores)
            mainstream_values = {
                "Innovator": 2, "Early Adopter": 3, "Early Majority": 5,
                "Late Majority": 5, "Laggard": 2
            }
            
            adopter_scores = []
            for idx in candidate_indices:
                user = self.user_data.iloc[idx]
                score = mainstream_values.get(user['adopter_category'], 0)
                adopter_scores.append(score)
            
            adopter_scores = np.array(adopter_scores)
            adopter_boost = np.clip((adopter_scores - 2) / 5, 0, 0.3)
            sim_boost += adopter_boost
        
        # 3. Income-based boost (up to 0.3)
        if any(term in query_lower for term in ["high income", "affluent", "premium", "high budget", 
                                              "luxury", "high-end", "wealthy"]):
            print("Applying high income boost")
            incomes = []
            for idx in candidate_indices:
                user = self.user_data.iloc[idx]
                incomes.append(user['annual_income'])
            
            incomes = np.array(incomes)
            # Boost higher incomes more (assume 75k is high income threshold)
            income_boost = np.clip((incomes - 75000) / 100000, 0, 0.3)
            sim_boost += income_boost
        
        elif any(term in query_lower for term in ["budget", "affordable", "low cost", "inexpensive", 
                                                "cost sensitive", "economical", "value"]):
            print("Applying budget-conscious boost")
            incomes = []
            for idx in candidate_indices:
                user = self.user_data.iloc[idx]
                incomes.append(user['annual_income'])
            
            incomes = np.array(incomes)
            # For budget queries, boost LOWER incomes (below 50k)
            income_boost = np.clip((50000 - incomes) / 50000, 0, 0.3) 
            sim_boost += income_boost
        
        # 4. Technology interest exact match boost (0.2 fixed boost)
        common_tech_interests = ["AI", "artificial intelligence", "machine learning", "IoT", 
                               "internet of things", "smart home", "VR", "virtual reality",
                               "AR", "augmented reality", "mobile", "cloud", "gaming", 
                               "robotics", "automation", "cryptocurrency", "blockchain", 
                               "wearable", "smartphone", "5G", "cyber security"]
                               
        for tech_term in common_tech_interests:
            if tech_term.lower() in query_lower:
                print(f"Applying boost for '{tech_term}' interest match")
                for i, idx in enumerate(candidate_indices):
                    user = self.user_data.iloc[idx]
                    if tech_term.lower() in user['tech_interest'].lower():
                        sim_boost[i] += 0.2  # Fixed boost for exact tech interest match
        
        # Apply the boost to similarity scores
        for i, idx in enumerate(candidate_indices):
            # Apply boost only to the candidates we're considering
            similarities[candidate_indices[i]] += sim_boost[i]
        
        # Re-rank based on boosted scores
        top_indices = np.argsort(similarities)[::-1][:top_n]
        
        # Format results
        predictions = []
        for idx in top_indices:
            user = self.user_data.iloc[idx]
            info = self.user_descriptions[idx]
            
            result = {
                'user_id': user['user_id'],
                'User': info,
                'Similarity': float(similarities[idx])
            }
            predictions.append(result)
        
        return {"predictions": predictions}
    
    @classmethod
    def log_model(cls, model_name, embeddings_path, tech_dataset_path, demo_dir=None):
        """
        Logs the model to MLflow with appropriate artifacts and schema.
        """
        # Check if the files exist
        for path in [embeddings_path, tech_dataset_path]:
            if not os.path.exists(path):
                raise FileNotFoundError(f"File not found: {path}")
        
        # Print file sizes for information
        emb_size = os.path.getsize(embeddings_path) / (1024 * 1024)
        tech_size = os.path.getsize(tech_dataset_path) / (1024 * 1024)
        
        print(f"Embeddings file size: {emb_size:.2f} MB")
        print(f"Tech dataset file size: {tech_size:.2f} MB")
        
        # Simple input schema - just accepting a query string
        input_schema = Schema([ColSpec("string", "query")])
        
        # Output schema now matches the HTML expectation structure
        output_schema = Schema([
            TensorSpec(np.dtype("object"), (-1,), "predictions")
        ])
        
        # Parameters schema - include show_score to match HTML interface
        params_schema = ParamSchema([
            ParamSpec("top_n", "integer", 5),
            ParamSpec("show_score", "boolean", True)
        ])
        
        # Define model signature
        signature = ModelSignature(inputs=input_schema, outputs=output_schema, params=params_schema)
        
        # Define necessary package requirements - adding sentence-transformers
        requirements = [
            "scikit-learn",
            "pandas",
            "numpy",
            "tabulate",
            "torch",
            "transformers",
            "sentence-transformers"
        ]
        
        # Define artifacts dictionary
        artifacts = {
            "embeddings_path": embeddings_path,
            "tech_dataset_path": tech_dataset_path
        }
        
        # Add demo directory to artifacts if provided and exists
        if demo_dir and os.path.exists(demo_dir):
            artifacts["demo"] = demo_dir
            
        # Define metadata with demo template if demo directory is provided and has index.html
        metadata = {}
        if demo_dir and os.path.exists(os.path.join(demo_dir, "index.html")):
            metadata["demo_template"] = "demo/index.html"
        
        # Log the model in MLflow
        mlflow.pyfunc.log_model(
            model_name,
            python_model=cls(),
            artifacts=artifacts,
            signature=signature,
            pip_requirements=requirements,
            metadata=metadata
        )

## MLflow Model Deployment

The following function handles the MLflow experiment setup, model logging, and registration. 
MLflow is used to:
- Track experiments and model versions
- Package the model with its dependencies and artifacts
- Register the model in the Model Registry for deployment
- Store the UI components for interactive demos

In [4]:
def log_model_to_mlflow():
    # Set the MLflow experiment name
    experiment_name = "Tech Adoption Similarity"
    mlflow.set_experiment(experiment_name=experiment_name)
    print(f"Experiment name: {experiment_name}")
    
    # Check if demo directory exists and has index.html
    demo_dir = "demo"
    index_html_path = os.path.join(demo_dir, "index.html")
    
    if os.path.exists(index_html_path):
        print(f"Found UI at {index_html_path}, will include in model deployment")
    else:
        print(f"Warning: UI file not found at {index_html_path}")
        os.makedirs(demo_dir, exist_ok=True)
        print("Creating demo directory...")

    # Start an MLflow run
    with mlflow.start_run(run_name="Tech_Adoption_Similarity_Run") as run:
        # Print the artifact URI for reference
        print(f"Run's Artifact URI: {run.info.artifact_uri}")
        
        # Log the tech adoption similarity model to MLflow
        model_name = "Tech_Adoption_Similarity"
        TechAdoptionSimilarityModel.log_model(
            model_name=model_name,
            embeddings_path="data/tech_embeddings.npy",
            tech_dataset_path="data/tech_adoption_dataset.csv",
            demo_dir=demo_dir if os.path.exists(demo_dir) else None
        )

        # Register the logged model in MLflow Model Registry
        registered_model = mlflow.register_model(
            model_uri=f"runs:/{run.info.run_id}/{model_name}", 
            name=model_name
        )
        
        # Get the version number of the registered model
        version = registered_model.version
        print(f"Registered model: {model_name}")
        print(f"Model version: {version}")
        
        return run.info.run_id

## Inference Pipeline

The `find_similar_users` function demonstrates how to load our deployed model and use it for inference. This represents what would happen in a production environment when the model is called through an API or application interface.

In [5]:
def find_similar_users(query, run_id=None, top_n=5):
    """
    Find similar tech users for a given query.
    
    Args:
        query (str): The search query (e.g., "Find users with high tech interest and early adoption patterns")
        run_id (str, optional): MLflow run ID. If None, uses the latest model version
        top_n (int): Number of results to return
        
    Returns:
        DataFrame: Similar tech users
    """
    if run_id:
        # Load model from specific run
        model_uri = f"runs:/{run_id}/Tech_Adoption_Similarity"
    else:
        # Get latest model version
        client = MlflowClient()
        model_metadata = client.get_latest_versions("Tech_Adoption_Similarity", stages=["None"])
        latest_model_version = model_metadata[0].version
        model_uri = f"models:/Tech_Adoption_Similarity/{latest_model_version}"
    
    # Load the model
    model = mlflow.pyfunc.load_model(model_uri)
    
    # Prepare simple input data
    input_data = {"query": [query]}
    
    # Run inference
    result = model.predict(input_data)
    
    # Extract predictions array from the result
    predictions = result.get("predictions", [])
    
    # Convert to DataFrame for better display
    return pd.DataFrame(predictions)

## Interactive Demo

The `run_demo` function shows a complete end-to-end demonstration, from model deployment to user search. This simulates how a product manager would interact with the system in a real environment.

The demo covers:
1. Model deployment to MLflow
2. Running a sample query 
3. Displaying formatted results
4. Error handling for failed queries

In [6]:
def run_demo():
    # Log model to MLflow
    run_id = log_model_to_mlflow()
    
    if not run_id:
        print("Model logging failed.")
        return
    
    # Use a sample query
    query = "Find users with high tech interest who are early adopters of new technologies"
    
    try:
        # Get similar users based on the query text
        similar_users = find_similar_users(
            query=query,
            run_id=run_id,
            top_n=5
        )
        
        # Display results
        print(f"\nQuery: {query}")
        print("\nTop similar tech users:")
        print(tabulate(similar_users, headers='keys', tablefmt='fancy_grid', showindex=False))
    
    except Exception as e:
        print(f"Error during inference: {e}")
        print("Displaying sample of the tech dataset instead:")
        tech_df = pd.read_csv("data/tech_adoption_dataset.csv", nrows=5)
        print(tabulate(tech_df, headers='keys', tablefmt='fancy_grid', showindex=False))

run_demo()

Experiment name: Tech Adoption Similarity
Found UI at demo/index.html, will include in model deployment
Run's Artifact URI: /phoenix/mlflow/970250650912509670/79fc657690004b578f1081a812336067/artifacts
Embeddings file size: 1.46 MB
Tech dataset file size: 0.87 MB


Registered model 'Tech_Adoption_Similarity' already exists. Creating a new version of this model...
Created version '31' of model 'Tech_Adoption_Similarity'.


Registered model: Tech_Adoption_Similarity
Model version: 31
Loaded embeddings shape: (1000, 384)
Loaded tech data shape: (1000, 29)
SentenceTransformer model 'sentence-transformers/all-MiniLM-L6-v2' loaded successfully
Processing query: 'Find users with high tech interest who are early adopters of new technologies'
Applying early adopter boost
Applying boost for 'AR' interest match

Query: Find users with high tech interest who are early adopters of new technologies

Top similar tech users:
╒═══════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤══════════════╕
│ user_id   │ User                                                                                                                                                                                                                     

## Next Steps and Extensions

This model could be extended in several ways:

1. **Feature Enhancement**:
   - Add support for filtering by specific technology categories
   - Incorporate product adoption history for more nuanced matching
   - Include platform preferences and brand loyalty data

2. **Performance Optimization**:
   - Use vector databases (like FAISS or Pinecone) for faster similarity search at scale
   - Implement batch processing for large user bases

3. **User Experience**:
   - Develop a more advanced UI with result filtering and sorting
   - Add visualizations of user segments
   - Implement feedback mechanisms to improve recommendations

4. **Integration**:
   - Connect with CRM systems for seamless workflow integration
   - Set up automated alerts when high-value opportunities are identified
   - Create scheduled reports based on specific queries

By deploying this system, tech companies can move from static, rule-based user segmentation to dynamic, semantically-rich user discovery that adapts to product managers' specific needs.